In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Selenium 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.set_page_load_timeout(180)  # 3분 페이지 로드 대기
driver.implicitly_wait(10)  # 암묵적 대기

# 타겟 URL
url = "https://www.rottentomatoes.com/m/avatar/reviews?type=user"
driver.get(url)

# 리뷰 데이터 저장 리스트
reviews_data = []

# Load More 버튼 반복 클릭
try:
    while True:
        try:
            load_more_button = driver.find_element(By.CSS_SELECTOR, "rt-button")
            if load_more_button.is_displayed():
                driver.execute_script("arguments[0].click();", load_more_button)
                time.sleep(10)  # 충분한 대기 시간
            else:
                print("No more 'Load More' button.")
                break
        except Exception as e:
            print(f"Error clicking 'Load More': {e}")
            break

    # 리뷰 항목 크롤링
    review_items = driver.find_elements(By.CLASS_NAME, "audience-review-row")

    for item in review_items:
        reviewer = item.find_element(By.CLASS_NAME, "audience-reviews__name").text if item.find_elements(By.CLASS_NAME, "audience-reviews__name") else "Anonymous"
        date = item.find_element(By.CLASS_NAME, "audience-reviews__duration").text if item.find_elements(By.CLASS_NAME, "audience-reviews__duration") else "Unknown"
        text = item.find_element(By.CLASS_NAME, "audience-reviews__review js-review-text").text if item.find_elements(By.CLASS_NAME, "audience-reviews__review js-review-text") else "No review text"
        rating = item.find_element(By.TAG_NAME, "rating-stars-group").get_attribute("score") if item.find_elements(By.TAG_NAME, "rating-stars-group") else "No rating"

        reviews_data.append({
            "Reviewer": reviewer,
            "Date": date,
            "Review": text,
            "Rating": rating,
        })

except Exception as e:
    print(f"An error occurred during scraping: {e}")

finally:
    # 드라이버 종료 및 데이터 저장
    driver.quit()

    if reviews_data:
        df = pd.DataFrame(reviews_data)
        df.to_csv("rottentomatoes_avatar_partial_reviews.csv", index=False)
        print("Partial reviews have been saved.")
    else:
        print("No reviews were successfully scraped.")
